# Using Environ for Solvation Effects on Isolated Systems

## Tutorial Setup

Import all the Python modules needed in this tutorial. Most of these are very common in scientific computing, some are popular tools in atomistic simulations.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
#
# ASE is a very convenient module for setting up simulations on molecules and 
# bulk materials
#
from ase.io import read
from ase.io.cube import read_cube
from ase.visualize import view

# Modified version of ASE's write_cube function which also writes atomic charges.
# Original version always assigns zero to atomic charges.
from ase_cube_mod import write_cube

## Modeling a single molecule in solution

Probably the most straightforward application of continuum embedding is modeling the solvation of a single molecule in infinite dilution. In this tutorial, we will use a single water molecule as our solute. We will use the popular TIP3P model of water for the molecular geometry and atomic charges. We have already prepared everything in the file tip3p.xyz.

In [ ]:
atoms = read('tip3p.xyz')
view(atoms, viewer='x3d')

The .xyz file defines only a single isolated molecule. However, Environ internally uses a formalism for periodic systems, so we need to put our molecule in a periodic simulation cell. For simplicity, but also to ensure a better behaviour of the electrostatics interactions, a cubic cell is the best choice to approximate isolated systems. 

In [ ]:
atoms.set_cell(15. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

We can now write the first of two input files for Environ, which will contain all information about the solute and the simulation cell. Environ uses the .cube file format for this purpose. This format contains the simulation cell and atomic positions and charges. 

Additionally, it can also contain a continuous electron density on a grid which fills the entire cell. For ths tutorial, we use only the atomic charges. We tell Environ to read only the cell information and atomic charges and positions while ignoring the grid information and volumetric data by passing only a single data point.

(NB: When using Environ self-consistently with DFT, it will receive this information from the DFT program. Only the second input file environ.in, which we discuss further below, is needed in that case.)

In [ ]:
write_cube(open('tip3p.cube', 'w'), atoms, [[[0.]]])

With the solute defined, we can now turn our attention to the solvent. The solvent model as well as all numerical methods and parameters are defined in the input file environ.in. For this tutorial, we will use the soft-sphere continuum solvation (SSCS) model with standard parameters for water as a solvent.

Additionally, we will use a periodic boundary correction which removes the spurious interaction of the solute with its periodic images in neighboring cells (Environ needs periodic boundary conditions purely for numerical reasons. We are, in this example, not actually interested in a periodic system, so we need to remove periodic interactions.)

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 4  ! Write out boundary function, potential, etc. Slows down
                ! Environ noticeably and can take a lot of disk space.
                ! Only use this when you actually need it!
   !
   env_electrostatic = .TRUE.
   environ_type = 'water'
   env_ecut = 100.  ! Defines grid density; higher value = denser grid
   !
/
&BOUNDARY
   !
   solvent_mode = 'ionic'  ! Define cavity based on atomic positions (=SSCS) 
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0  ! 0 = isolated molecule, 2 = surface slab
   !
   tol = 1.d-2
   !
/
EOF

That's all we need to set up our first Environ calculation!

In [ ]:
%%bash
export environ_bin=/work/data/programs_new/Environ/programs/driver
orterun -n 4 $environ_bin -n from_cube -c tip3p.cube --no-density | tee environ.out # --force to also compute solvent contribution to atomic forces

## Visualizing results

Above, we set Environ to write some quantities like the boundary function and the solvent potential to cube files. We can now visualize them.

In [ ]:
cube_boundary = read_cube(open('boundary_solvent.cube', 'r'))

# Get 2D cross section. Because we centered the molecule in the cell, we can simply take the central slice of the 3D data
data_shape = cube_boundary['data'].shape
center_ind = int(data_shape[2]/2)
crosssec_boundary = cube_boundary['data'][:,:,center_ind].T

# Create 2D grid to plot
x = np.linspace(0., cube_boundary['atoms'].cell[0,0], data_shape[0]+1)[:-1]
y = np.linspace(0., cube_boundary['atoms'].cell[1,1], data_shape[1]+1)[:-1]
X, Y = np.meshgrid(x, y)

# Plot cross section
plt.pcolormesh(X, Y, crosssec_boundary, cmap='bwr', vmin=0., vmax=2., rasterized=True)

# Draw simple ball and stick model of molecule
pos = atoms.get_positions()
colors = ['red', 'grey', 'grey']
radii = [100., 50., 50.]
plt.plot([pos[0,0], pos[1,0]], [pos[0,1], pos[1,1]], '-k', linewidth=3.)
plt.plot([pos[0,0], pos[2,0]], [pos[0,1], pos[2,1]], '-k', linewidth=3.)
plt.scatter(pos[:,0], pos[:,1], c=colors, marker='o', zorder=1000, edgecolors='k', s=radii)

plt.title('Boundary function')
plt.gca().set_aspect('equal')
plt.axis('off')

In [ ]:
cube_pot = read_cube(open('dvtot.cube', 'r'))

# Get 2D cross section. Because we centered the molecule in the cell, we can simply take the central slice of the 3D data
crosssec_pot = cube_pot['data'][:,:,center_ind].T
vmax = np.max(np.abs(crosssec_pot))

# Plot cross section; Environ internally uses the sign convention from electronic structure theory where
# electrons are positive, and their potentials accordingly. Flip sign here for more conventional image.
plt.pcolormesh(X, Y, -crosssec_pot, cmap='seismic', vmin=-vmax, vmax=vmax, rasterized=True)

# Draw simple ball and stick model of molecule
plt.plot([pos[0,0], pos[1,0]], [pos[0,1], pos[1,1]], '-k', linewidth=3.)
plt.plot([pos[0,0], pos[2,0]], [pos[0,1], pos[2,1]], '-k', linewidth=3.)
plt.scatter(pos[:,0], pos[:,1], c=colors, marker='o', zorder=1000, edgecolors='k', s=radii)

plt.title('Solvent potential')
plt.gca().set_aspect('equal')
plt.axis('off')